In [24]:
import os;
import json;
import builtins
import pandas as pd;
import duckdb

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
op = ['join', 'merge']
#datasets = ['fb', 'wiki', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal', 'osm', 'books']
#threads = ["1", "4", "16", "32"]
datasets = ['str']
threads = ["1", "4", "16", "32"]
for op in op:
    for dataset in datasets:
        for thread in threads:
            testdir = os.path.join(dir, "_".join([op, dataset, thread]), "outputs", "results")
            if (not os.path.exists(testdir)):
                continue
            rundirs = os.listdir(testdir)
            for rundir in rundirs:
                for test_result_file in os.listdir(os.path.join(testdir, rundir)):
                    json_file = builtins.open(os.path.join(testdir, rundir, test_result_file))
                    test_result = json.load(json_file)
                    test_result['run'] = rundir
                    test_result['dataset'] = dataset
                    test_result['op'] = op
                    if dataset == 'osm' or dataset == 'books':
                        test_result['input_size'] = 800_000_000 
                    elif dataset == 'str':
                        test_result['input_size'] = 100_000_000
                    else:
                        test_result['input_size'] = 200_000_000 
                    test_results.append(test_result)
                    json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)
test_dataframe = test_dataframe.dropna(subset=['command'])

Index(['command', 'run', 'dataset', 'op', 'input_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key',
       'spec.index.leaf_size_in_pages', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


In [29]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "btree256": 256,
    "btree512": 512,
    "btree1024": 1024,
    "btree1048": 1024,
    "btree2048": 2048,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
indexType = {
    "pgm64": "PGM",
    "pgm128": "PGM",
    "pgm256": "PGM",
    "pgm512": "PGM",
    "pgm1024": "PGM",
    "pgm2048": "PGM",
    "btree256": "BTREE",
    "btree512": "BTREE",
    "btree1024": "BTREE",
    "btree1048": "BTREE",
    "btree2048": "BTREE",
    "hashjoin": "NA",
    "btree": 0,
    "sj": "SJ",
    "sj2": "NA",
    "standard_merge": "SM"
};
dataset_type = {
    "fb": "REAL",
    "wiki": "REAL",
    "osm": "REAL",
    "books": "REAL",
    "uniform_dense": "SYNTHETIC",
    "uniform_sparse": "SYNTHETIC",
    "normal": "SYNTHETIC",
    "lognormal": "SYNTHETIC",
    "str": "SYNTHETIC"
};
dataset_display = {
    "fb": "fb",
    "wiki": "wiki",
    "osm": "osm",
    "books": "books",
    "uniform_dense": "udense",
    "uniform_sparse": "usparse",
    "normal": "normal",
    "lognormal": "lognormal",
    "str": "16 Byte Strings"
};
dataset_order = {
    "fb": 0,
    "wiki": 1,
    "osm": 2,
    "books": 3,
    "uniform_dense": 4,
    "uniform_sparse": 5,
    "normal": 6,
    "lognormal": 7,
};

test_dataframe["threads"] = test_dataframe["spec.num_threads"]
test_dataframe["duration_sec"] = test_dataframe["result.duration_ns"] / (1000000000)
test_dataframe["ratio"] = test_dataframe["spec.common_key"]
test_dataframe["algo"] = test_dataframe["spec.algo_name"]
test_dataframe["inner_bytes_fetched"] = test_dataframe["result.inner_total_bytes_fetched"]
test_dataframe["epsilon"] = test_dataframe["spec.algo_name"].map(lambda x: epsilon[str(x).lower()])
test_dataframe["index_type"] = test_dataframe["spec.algo_name"].map(lambda x: indexType[str(x).lower()])
test_dataframe["index_name"] = test_dataframe["spec.algo_name"]
test_dataframe["dataset_type"] = test_dataframe["dataset"].map(lambda x: dataset_type[str(x).lower()])

results = test_dataframe[["dataset", "input_size", "dataset_type", "threads", "ratio", "op", "algo", "index_name", "index_type", "epsilon", "duration_sec", "inner_bytes_fetched"]]
results


,dataset,input_size,dataset_type,threads,ratio,op,algo,index_name,index_type,epsilon,duration_sec,inner_bytes_fetched
3,str,100000000.0,SYNTHETIC,1.0,5.0,join,btree1024,btree1024,BTREE,1024,16.654654,1.600029e+09
4,str,100000000.0,SYNTHETIC,1.0,5.0,join,btree256,btree256,BTREE,256,18.127506,1.600004e+09
5,str,100000000.0,SYNTHETIC,1.0,5.0,join,btree256,btree256,BTREE,256,17.779150,1.600004e+09
6,str,100000000.0,SYNTHETIC,1.0,10.0,join,pgm256,pgm256,PGM,256,2.839688,1.599984e+09
7,str,100000000.0,SYNTHETIC,1.0,5.0,join,pgm256,pgm256,PGM,256,4.856526,1.600008e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
3974,str,100000000.0,SYNTHETIC,4.0,5.0,merge,pgm2048,pgm2048,PGM,2048,3.107213,3.200004e+09
3975,str,100000000.0,SYNTHETIC,4.0,50.0,merge,pgm1024,pgm1024,PGM,1024,2.663677,3.200006e+09
3976,str,100000000.0,SYNTHETIC,4.0,100.0,merge,standard_merge,standard_merge,SM,0,3.116753,3.200006e+09
3977,str,100000000.0,SYNTHETIC,4.0,100.0,merge,pgm256,pgm256,PGM,256,2.654987,3.200006e+09


In [31]:
# Average results by time.
input_size = 100_000_000
rows = duckdb.sql(
    " SELECT ratio, epsilon, op, index_type, AVG(duration_sec) as v FROM results "
    f" WHERE threads = 1 AND input_size={input_size} AND op='join' AND index_type!='NA'"
    " GROUP BY ratio, threads, op, input_size, index_type, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
p = (duckdb.sql("PIVOT rows ON index_type,epsilon USING avg(v) GROUP BY ratio ORDER BY RATIO"))
print(p.df()[['ratio', 'SJ_0', 'BTREE_256', 'BTREE_2048', 'PGM_256', 'PGM_2048']].to_latex(index=False, float_format='%.3f'))

\begin{tabular}{rrrrrr}
\toprule
ratio & SJ_0 & BTREE_256 & BTREE_2048 & PGM_256 & PGM_2048 \\
\midrule
1.000 & 11.372 & 107.323 & 83.056 & 24.924 & 27.118 \\
5.000 & 3.157 & 17.953 & 14.105 & 4.683 & 5.046 \\
10.000 & 2.203 & 10.051 & 7.921 & 2.833 & 3.344 \\
50.000 & 1.314 & 2.416 & 1.924 & 0.847 & 0.882 \\
100.000 & 1.184 & 1.392 & 1.435 & 0.717 & 0.704 \\
\bottomrule
\end{tabular}



In [33]:
rows = duckdb.sql(
    " SELECT ratio, threads, epsilon, op, index_name, AVG(duration_sec) as v FROM results "
    " WHERE (ratio=10 or ratio=50 or ratio=100) AND input_size=100000000 AND op='join' AND index_type!='NA'"
    " AND (index_name='sj' OR epsilon=2048)"
    " GROUP BY ratio, threads, op, input_size, index_name, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
rows
print(duckdb.sql("PIVOT rows ON index_name USING AVG(v) GROUP BY threads,ratio ORDER BY threads,ratio").df().to_latex(index=False, float_format='%.3f'))

\begin{tabular}{rrrrr}
\toprule
threads & ratio & btree2048 & pgm2048 & sj \\
\midrule
1.000 & 10.000 & 7.921 & 3.344 & 2.203 \\
1.000 & 50.000 & 1.924 & 0.882 & 1.314 \\
1.000 & 100.000 & 1.435 & 0.704 & 1.184 \\
4.000 & 10.000 & 2.477 & 1.072 & 0.713 \\
4.000 & 50.000 & 0.674 & 0.272 & 0.443 \\
4.000 & 100.000 & 0.376 & 0.194 & 0.337 \\
8.000 & 10.000 & 1.651 & 0.689 & 0.548 \\
8.000 & 50.000 & 0.395 & 0.221 & 0.324 \\
8.000 & 100.000 & 0.269 & 0.151 & 0.280 \\
32.000 & 10.000 & 0.786 & 0.489 & 0.498 \\
32.000 & 50.000 & 0.289 & 0.236 & 0.277 \\
32.000 & 100.000 & 0.233 & 0.192 & 0.244 \\
\bottomrule
\end{tabular}



In [35]:
rows = duckdb.sql(
    " SELECT ratio, epsilon, op, index_type, AVG(duration_sec) as v FROM results "
    " WHERE threads = 1 AND input_size=100000000 AND op='merge' AND index_type!='NA'"
    " GROUP BY ratio, threads, op, input_size, index_type, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
p = (duckdb.sql("PIVOT rows ON index_type,epsilon USING avg(v) GROUP BY ratio ORDER BY RATIO"))
print(p)
print(p.df()[['ratio', 'SM_0', 'BTREE_256', 'BTREE_2048', 'PGM_256', 'PGM_2048']].to_latex(index=False, float_format='%.3f'))

┌────────┬─────────┬────────────────────┬────────────────────┬───┬────────────────────┬─────────┬─────────┬────────┐
│ ratio  │ BTREE_0 │     BTREE_1024     │     BTREE_2048     │ … │        SM_0        │ SM_1024 │ SM_2048 │ SM_256 │
│ double │ double  │       double       │       double       │   │       double       │ double  │ double  │ double │
├────────┼─────────┼────────────────────┼────────────────────┼───┼────────────────────┼─────────┼─────────┼────────┤
│    1.0 │    NULL │ 3.6445897333333335 │ 3.4949636130000004 │ … │        4.200778664 │    NULL │    NULL │   NULL │
│    2.0 │    NULL │        3.174356543 │ 3.0832515753333336 │ … │  4.157332778333333 │    NULL │    NULL │   NULL │
│    5.0 │    NULL │ 17.858731408333337 │ 15.152702550333338 │ … │ 5.2472953586666655 │    NULL │    NULL │   NULL │
│   10.0 │    NULL │ 11.098453535333332 │        9.608938701 │ … │  4.685297184999998 │    NULL │    NULL │   NULL │
│   50.0 │    NULL │  4.550107063000001 │  4.232528185333333 │ …

In [ ]:
rows = duckdb.sql(
    " SELECT ratio, threads, epsilon, op, index_name, AVG(duration_sec) as v FROM results "
    " WHERE (ratio=10 or ratio=50 or ratio=100) AND input_size=200000000 AND op='merge' AND index_type!='NA'"
    " AND (index_name='standard_merge' OR epsilon=2048)"
    " GROUP BY ratio, threads, op, input_size, index_name, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
rows
print(duckdb.sql("PIVOT rows ON index_name USING AVG(v) GROUP BY threads,ratio ORDER BY threads,ratio").df().to_latex(index=False, float_format='%.3f'))

\begin{tabular}{rrrr}
\toprule
threads & ratio & btree2048 & pgm2048 \\
\midrule
1 & 10 & 6.245 & 4.826 \\
1 & 50 & 3.518 & 3.243 \\
1 & 100 & 3.117 & 2.979 \\
4 & 10 & 2.702 & 2.647 \\
4 & 50 & 2.500 & 2.641 \\
4 & 100 & 2.532 & 2.436 \\
16 & 10 & 2.722 & 2.705 \\
16 & 50 & 2.550 & 2.514 \\
16 & 100 & 2.509 & 2.491 \\
32 & 10 & 2.900 & 2.949 \\
32 & 50 & 2.703 & 2.767 \\
32 & 100 & 2.749 & 2.730 \\
\bottomrule
\end{tabular}



In [ ]:
def group_by_threads(op, ratio):
    rows = duckdb.sql(
        " SELECT ratio,index_name,threads, MEDIAN(duration) as v"
        f" FROM results where op='{op}' AND ratio={ratio}  AND index_type!='NA'"
        " GROUP BY dataset, ratio, index_name, threads "
        " ORDER BY Threads")
    return duckdb.sql('PIVOT rows ON index_name USING MEDIAN(v)')
ratios = [10, 50, 100]
for ratio in ratios:
    for op in ['join', 'merge']:
        group_by_threads(op, ratio).df().to_csv('_'.join([op, str(ratio), 'threadStudy.csv']))

## PGM Index Memory Study

In [ ]:
def compare_btree_pgm_memory(epsilon, thread, dataset_type):
        where = f"WHERE threads={thread} AND epsilon={epsilon} AND ratio=10 "
        if dataset_type != "all":
                where = where + f" AND dataset_type='{dataset_type}' "
        query = ("SELECT ratio, threads, algo, index_type, dataset, dataset_type,"
                "epsilon, MEAN(index_memory) as memory_mean, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon, dataset, dataset_type) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING MEAN(memory_mean) GROUP BY ratio,dataset").df()
        df['REL'] = (df['PGM']) / df['BTREE']
        return df

def compare_btree_pgm_build_duration(epsilon, thread, dataset_type):
        where = f"WHERE threads={thread} AND epsilon={epsilon} AND ratio=10 "
        if dataset_type != "all":
                where = where + f" AND dataset_type='{dataset_type}' "
        query = ("SELECT ratio, threads, algo, index_type, dataset, dataset_type,"
                "epsilon, MEAN(index_build_duration) as index_build_duration, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon, dataset, dataset_type) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING MEAN(index_build_duration) GROUP BY ratio,dataset").df()
        df['REL'] = (df['PGM']) / df['BTREE']
        return df
dict = []
dict.append({
        'epsilon':256, 
        'ALL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='all')['PGM'].mean(),
        })
dict.append({
        'epsilon':1024, 
        'ALL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='all')['PGM'].mean(),
        })
dict.append({
        'epsilon': 2048, 
        'ALL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='all')['PGM'].mean(),
        })

print(pd.DataFrame(dict).to_latex(index=False))

dict = []
dict.append({
        'epsilon':256, 
        'ALL': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
dict.append({
        'epsilon':1024, 
        'ALL': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
dict.append({
        'epsilon': 2048, 
        'ALL': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
print(pd.DataFrame(dict).to_latex(index=False))

## PGM vs TREE SJ same epsilon (Duration)

In [ ]:
def compare_btree_pgm_duration(epsilon, thread, dataset, op):
        where = f"WHERE threads={thread} AND op='{op}' AND (epsilon={epsilon} OR algo='sj' OR algo='standard_merge')"
        if dataset != "all":
                where = where + f"AND dataset='{dataset}'"
        query = ("SELECT ratio, threads, algo, index_type,"
                "epsilon, MEDIAN(duration) as duration_mean, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        print(subresult)
        df = duckdb.sql("PIVOT subresult ON index_type USING sum(duration_mean) GROUP BY ratio").df()
        print(df)
        baseline = 'SJ' if op == 'join' else 'SM'
        df['RATIO'] = df['ratio']
        if 'PGM' in df.columns:
                df['PGM_REL'] = (df['PGM']/ df[baseline])
        if 'BTREE' in df.columns:
                df['BTREE_REL'] = (df['BTREE']/ df[baseline])
        return df

def compare_pgm_bytes_fetched(epsilon, thread, dataset, op):
        where = f"WHERE threads={thread} AND op='{op}' AND (epsilon={epsilon} OR algo='sj' OR algo='standard_merge')"
        if dataset != "all":
                where = where + f"AND dataset='{dataset}'"
        query = ("SELECT ratio, threads, algo, index_type,"
                "epsilon, MEDIAN(inner_bytes_fetched) as median, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING AVG(median) GROUP BY ratio").df()
        baseline = 'SJ' if op == 'join' else 'SM'
        df['RATIO'] = df['ratio']
        df['BTREE_REL'] = (df['BTREE']/ df[baseline])
        df['PGM_REL'] = (df['PGM']/ df[baseline])
        return df


In [ ]:

df = pd.DataFrame();
df['ratio'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['RATIO'])
df['t=1'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'])
df['t=4'] = (compare_btree_pgm_duration(epsilon=2048, thread=4, dataset='all', op='join')['PGM_REL'])
df['t=16'] = (compare_btree_pgm_duration(epsilon=2048, thread=16, dataset='all', op='join')['PGM_REL'])
df['t=32'] = (compare_btree_pgm_duration(epsilon=2048, thread=32, dataset='all', op='join')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='join')['RATIO'])
df['t=1'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'])
df['t=4'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=4, dataset='all', op='join')['PGM_REL'])
df['t=16'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=16, dataset='all', op='join')['PGM_REL'])
df['t=32'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=32, dataset='all', op='join')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['RATIO'])
df['t=1'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'])
df['t=4'] = (compare_btree_pgm_duration(epsilon=2048, thread=4, dataset='all', op='merge')['PGM_REL'])
df['t=16'] = (compare_btree_pgm_duration(epsilon=2048, thread=16, dataset='all', op='merge')['PGM_REL'])
df['t=32'] = (compare_btree_pgm_duration(epsilon=2048, thread=32, dataset='all', op='merge')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='merge')['RATIO'])
df['t=1'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'])
df['t=4'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=4, dataset='all', op='merge')['PGM_REL'])
df['t=16'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=16, dataset='all', op='merge')['PGM_REL'])
df['t=32'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=32, dataset='all', op='merge')['PGM_REL'])
print(df.to_latex(index=False))



In [ ]:
f = compare_btree_pgm_duration
df = []
df.append({'epsilon': 256, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='join')['PGM_REL'].mean()})
df.append({'epsilon': 1024, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=1024, thread=1, dataset='all', op='join')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'].mean()})

df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})
df.append({'epsilon': 1024, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=1024, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())

f = compare_btree_pgm_duration
df = []
df.append({'epsilon': 256, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})

df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())

In [ ]:
f = compare_btree_pgm_duration
df = []

df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})

df.append({'epsilon': 512, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=512, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})
df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())